In [3]:
# !git clone https://github.com/riteshsv/timeseries.git

In [4]:
import numpy as np
import os

In [5]:
import pandas as pd

In [6]:
import torch
from torch import nn     
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
def preparedata_time_window(input_data,window):
    out_data = []
    # rearrange the input in form of output based on 3 observations in past
    for i in range(len(input_data)):
        if (i+window+1) <= len(input_data):
            out_data.append((input_data[i:i+window],input_data[i+window:i+(window+1)]))
            
    return out_data

In [9]:
class StandardizeInput():
  def __init__(self):
    self.scaler = MinMaxScaler(feature_range=(-1,1)) 
  def __call__(self,data):
    return self.scaler.fit_transform(data)

# scaler2 = MinMaxScaler(feature_range=(-1,1))
# scaled_input2 = scaler2.fit_transform(data.reshape(-1,1))


In [10]:
class UnivariateTsDataset(Dataset):
    """Univariate time serices dataset."""

    def __init__(self, csv_file, target:str=None, transform=None, window=1,features:list=None,n_features=1,scaler=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            window (integer): Number of time steps in the past
            features (list) : list of features to be used, default is all features
            n_features (integer) : number of features to be used. default is 1, min number of features required
        """
        self.transform = transform
        self.target = target
        self.window = window
        self.features = features
        self.scaler=scaler
        if len(self.features) > 0:
          self.n_features = len(self.features)
        else:
          self.n_features = n_features
        self.rawdata = pd.read_csv(csv_file)
        self.data = self.preprocess(self.rawdata)
        
    
    def preprocess(self,input_df):
      if self.features != None:
        input_data = input_df[self.features]
      else:
        input_data = input_df
      # scaling
      if self.scaler != None:
        input_data = self.scaler(input_data)
      # convert to tensors
      input_data_tensor = torch.tensor(input_data,dtype=torch.float32,device=device).view(-1)
      out_data = []
      # rearrange the input in form of output based on 3 observations in past
      for i in range(len(input_data_tensor)):
        if (i+self.window+1) <= len(input_data_tensor):
          out_data.append((input_data_tensor[i:i+self.window],input_data_tensor[i+self.window:i+(self.window+1)]))
            
      return out_data
    
    def train_val_test_split(self,train_size=0.8,test_size=.20,val_size=0.0):
      train_len = int(train_size*len(self.data))
      train_set = torch.utils.data.Subset(self,np.arange(train_len))
      val_len = int(len(self.data)*val_size)
      if val_size > 0:
        val_set = torch.utils.data.Subset(self,np.arange(train_len,(train_len+val_len)))
      test_set = torch.utils.data.Subset(self,np.arange((train_len+val_len),len(self.data)))
      if val_size > 0:
        return train_set,val_set,test_set
      return train_set,test_set

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = self.data[idx]

        return sample

In [11]:
class LSTMSimple(nn.Module):
    def __init__(self, input_dim,hidden_dim,out_size,num_layers=1,batch_size=1,device=None):
        super(LSTMSimple,self).__init__()
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_dim, hidden_dim,num_layers=num_layers)
        self.out = nn.Linear(hidden_dim,out_size)
        self.hidden_dim = hidden_dim
        self.n_layers = num_layers
        if device == None:
          if torch.cuda.is_available():
            self.device = torch.device('cuda')
          else:
            self.device = torch.device('cpu')
        else:
          self.device = device
        self.hidden = (torch.zeros(1*self.n_layers,self.batch_size,self.hidden_dim,device=self.device),torch.zeros(1*self.n_layers,self.batch_size,self.hidden_dim,device=self.device))
    def forward(self,x):
      self.hidden = (torch.zeros(1*self.n_layers,self.batch_size,self.hidden_dim,device=self.device),torch.zeros(1*self.n_layers,self.batch_size,self.hidden_dim,device=self.device))
      out,self.hidden = self.lstm(x.view(len(x[1]),self.batch_size,-1),self.hidden)
      preds = self.out(out[-1])
      return preds[0]
    def predict(self,n_preds,past_val,batch_size=1):
      out = []
      with torch.no_grad():
        input_ten = torch.tensor(past_val,dtype=torch.float32,device=self.device)
        self.eval()
        for p in range(n_preds):
          input_ten = input_ten[-len(past_val):].view(-1,len(past_val))
          self.hidden = (torch.zeros(1*self.n_layers,batch_size,self.hidden_dim,device=self.device),
                         torch.zeros(1*self.n_layers,batch_size,self.hidden_dim,device=self.device))
          preds = model(input_ten)
          input_ten = torch.cat((input_ten.view(-1),preds),0)
          out.append(preds)
      return out
    # def predict(n_preds,input_ten):
    #   out = []
    #   with torch.no_grad():
    #     for p in range(n_preds):
    #       input_ten = input_ten[-6:].view(-1,6)
    #       # model.hidden = (torch.zeros(2,1,64,device=device),torch.zeros(2,1,50,device=device))
    #       preds = model(input_ten)
    #       input_ten = torch.cat((input_ten.view(-1),preds),0)
    #       actual_pred = dataset.scaler.scaler.inverse_transform(np.array(preds.view(-1).cpu()).reshape(-1,1))
    #       out.append(actual_pred)
    #   print(actual_pred)
    #   return out

In [12]:
class Trainer():
  def __init__(self,model,train_data,loss_fn,optimizer,val_data=None,epochs=1):
    self.model = model
    self.epochs = epochs
    self.train_data = train_data
    self.val_data = val_data
    self.loss_fn = loss_fn
    self.optimizer = optimizer
    self.predictions = []
  def train(self):
    for epoch in range(self.epochs):
      self.model.train()
      train_loss = 0.0  
      for x,t in self.train_data:
        self.model.zero_grad()
        preds = model(x)
        self.predictions.append(preds)
        loss = self.loss_fn(preds,t)
        loss.backward()
        self.optimizer.step()
        train_loss += loss.item()
      train_loss = train_loss/len(self.train_data)
      if self.val_data != None:
        val_loss = self.__validation()
      if (epoch+1)%10 == 0:
        print(f"Epoch: {epoch+1} loss: {train_loss} validation loss: {val_loss}")
  def __validation(self):
    self.model.eval()  
    val_loss = 0.0
    with torch.no_grad():
      for x,t in self.val_data:
        preds = self.model(x)
        self.predictions.append(preds)
        loss = self.loss_fn(preds,t)
        val_loss += loss.item()
    return val_loss/len(self.val_data)

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)
print(type(device))

cuda
<class 'torch.device'>


In [14]:
dataset = UnivariateTsDataset('/content/timeseries/ULTRACEMCO.NS.csv',window=6,features=['Close'],n_features=1,scaler=StandardizeInput())

In [15]:
train_set,val_set,test_set = dataset.train_val_test_split(0.60,0.20,0.20)


In [16]:
batch_size = 4
train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=False,drop_last=True)
val_loader = torch.utils.data.DataLoader(val_set,batch_size=batch_size,shuffle=False,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=False,drop_last=True)


In [17]:
for x,y in train_loader:
  print(x.shape)
  print(x.view(len(x[1]),4,-1).size(-1))
  print(x.view(len(x[1]),4,-1).shape)

torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size([4, 6])
1
torch.Size([6, 4, 1])
torch.Size(

In [18]:
df = pd.read_csv('/content/timeseries/ULTRACEMCO.NS.csv')
# df.columns=['sales']


In [19]:
model = LSTMSimple(1,128,1,num_layers=6,batch_size=batch_size,device=device)
model.to(device)

LSTMSimple(
  (lstm): LSTM(1, 128, num_layers=6)
  (out): Linear(in_features=128, out_features=1, bias=True)
)

In [20]:
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001)


Perform a train test split

In [21]:
trainer = Trainer(model,train_loader,loss_fn,optimizer,val_loader,200)

In [22]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 10 loss: 0.08853899219280316 validation loss: 0.0783866176304097
Epoch: 20 loss: 0.05149039645524075 validation loss: 0.03430838935795085
Epoch: 30 loss: 0.045953372286425695 validation loss: 0.03298376134262071
Epoch: 40 loss: 0.04290335866648497 validation loss: 0.0338443632790586
Epoch: 50 loss: 0.04076496701196043 validation loss: 0.035150654352037236
Epoch: 60 loss: 0.03905192628088924 validation loss: 0.03636427804303821
Epoch: 70 loss: 0.0375816498676108 validation loss: 0.03732974639569875
Epoch: 80 loss: 0.03627792214845411 validation loss: 0.038000022914881505
Epoch: 90 loss: 0.03510630659164033 validation loss: 0.03836644286881589
Epoch: 100 loss: 0.034043394090581894 validation loss: 0.038431113986007404
Epoch: 110 loss: 0.03306102777262115 validation loss: 0.038195243916561594
Epoch: 120 loss: 0.03211977861550016 validation loss: 0.037656294927728595
Epoch: 130 loss: 0.03116582024247489 validation loss: 0.03680487246310804
Epoch: 140 loss: 0.030124747472453035 valid

In [23]:
def test(test_data):
   model.eval()  
   test_loss = 0.0
   preds = []
   with torch.no_grad():
     for x,t in test_data:
       pred = model(x)
       loss = loss_fn(pred,t)
       test_loss += loss.item()
       preds.append(pred)
     print(test_loss/len(test_data))
   return preds

In [24]:
p = test(test_loader)

0.01905136412824504


In [25]:
p_a = []
for x in p:
  p_a.append(dataset.scaler.scaler.inverse_transform(np.array(x.view(-1).cpu()).reshape(-1,1)))

In [26]:
def predict(n_preds,past_val):
  out = []
  with torch.no_grad():
    input_ten = torch.tensor(past_val,dtype=torch.float32,device=device)
    model.eval()
    for p in range(n_preds):
      input_ten = input_ten[-len(past_val):].view(-1,len(past_val))
      # model.hidden = (torch.zeros(2,1,64,device=device),torch.zeros(2,1,50,device=device))
      preds = model(input_ten)
      input_ten = torch.cat((input_ten.view(-1),preds),0)
      actual_pred = dataset.scaler.scaler.inverse_transform(np.array(preds.view(-1).cpu()).reshape(-1,1))
      out.append(actual_pred)
      print(actual_pred)
  return out

In [32]:
actual = predict(10,dataset.rawdata['Close'].iloc[-10:].to_numpy().reshape(1,-1))

RuntimeError: ignored

In [29]:
dataset.rawdata['Close'].iloc[-10:].to_numpy().reshape(1,-1).shape

(1, 10)

In [30]:
ap = [x.reshape(1)[0] for x in actual]
ap


NameError: ignored

In [ ]:
df = pd.read_csv('/content/timeseries/ULTRACEMCO.NS.csv')

In [ ]:
df2 = pd.DataFrame(pd.date_range('2022-11-17',periods=10).date)
df2.columns = ['Date']
df2['ap'] = ap

In [ ]:
df3 = pd.concat([df,df2],axis=0)

In [ ]:
df4 = df3[-30:]


In [ ]:
df4['Date'] = pd.to_datetime(df4['Date'])

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sns.lineplot(x='Date',y='Close',data=df4)
sns.lineplot(x='Date',y='ap',data=df4)
plt.ylabel("Price")
plt.xticks(rotation = 20)
plt.show()

In [ ]:
torch.cuda.empty_cache()

# New Section

In [ ]:
dataset.rawdata['Close'].iloc[-6:]

In [ ]:
len(train_loader)